In [1]:
# Import libraries
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text

In [2]:
#equipa de limpeza teste
df_answers = pd.read_csv("Answers.csv", encoding="ISO-8859-1")
df_questions = pd.read_csv("Questions.csv", encoding="ISO-8859-1")
df_tags = pd.read_csv("Tags.csv", encoding="ISO-8859-1")

In [3]:
df_questions = ['CreationDate'] = pd.to_datetime(df['CreationDate'])
df_answers = ['CreationDate'] = pd.to_datetime(df['CreationDate'])

In [ ]:
df.dtypes

In [ ]:
queries = [
    {"description": "Retrieve all questions that have a score greater than 300", "query": ""},
    
    {"description": "Retrieve all answers from 2008", "query": ""},
    
    {"description": "Retrive all questions that have 2 tags", "query": ""},

    {"description": "Retrive all questions from 2015 about PDF that have more than 4 answers", "query": ""}

]

In [ ]:
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'score' : {'$gt' : 300}}
        result = collection_stackoverflow.find(query)

    elif "answers from 2008" in q["description"]:
        query = {'creationdate': {'$regex' : '^2008-.*'}}
        result = collection_stackoverflow.find(query)

    elif "2 tags" in q["description"]:
        query = {}
        result = 
    else:
        query = {}
        result = 
        
    end_time = time.time()
 
    list_result = list(result)
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {len(list_result)} found in {end_time - start_time:.4f} seconds")
client.close()

In [ ]:
mysql_times = []
with engine.connect() as connection:
    for q in queries:
        start_time = time.time()
        if "score greater" in q["description"]:
            result = connection.execute(text("SELECT title FROM questions WHERE score > 300"))

        elif "answers from 2008" in q["description"]:
            result = connection.execute(text("SELECT * FROM answers WHERE creationdate LIKE '2008%'"))
        
        elif "2 tags" in q["description"]:
            result = connection.execute(text("(SELECT id FROM tags GROUP BY id HAVING COUNT(tag) = 2) AS t JOIN questions q ON t.id = q.id"))
            
        else:
            result = connection.execute(text("SELECT title FROM questions WHERE creationdate LIKE '2015%' AND title LIKE '%PDF%' AND (SELECT COUNT(*) FROM answers WHERE parentid = questions.id) > 4")) #maybe we need a join to compare parentid to questions.id
        results =  result.fetchall()
        end_time = time.time()
        mysql_times.append(end_time - start_time)
        print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")

In [ ]:
comparison_df = pd.DataFrame({
    "Query": [q["description"] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MySQL_Time(s)": mysql_times
})

print(comparison_df)